# Capstone Project: IBM-Coursera Professional Data Science Specialization

#### Introduction
******
The actual study wants to pinpoint the living and socio-economic conditions in   every neighborhood around the metropolitan district of San Francisco de Quito. With the help of Geolocation tools and some data science techniques like clustering, we can classify Neighborhoods based on socio economic information of each neighborhood. The result of the study could help to answer some important question about the inequality around the city:
-	Which neighborhoods have better or worst living conditions (House construction features and basic services like drinking water and home waste management)? 
-	Difference of Education skills and conditions between neighborhoods
-	Teen pregnancy rates 
-	Branch and job category 
-	Neighborhoods with similar commercial characteristics
-	Ethnic distribution 


#### Import libraries

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 29 10:12:46 2020

@author: mateo
"""
# Cluster de parroquias en quito
##Importar parroquias a  dataframe
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
! conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

#### GEO DATA SAN FRANCISCO DE QUITO
Extract information from this resource: 
[LINK OF QUITO GEO DATA](http://www.codigopostalecuador.com/quito-1896)

In [2]:
req = requests.get("http://www.codigopostalecuador.com/quito-1896")

soup = BeautifulSoup(req.content,'lxml')

table= soup.find_all('table')
tr= soup.find_all('tr')
#Crear dataframe
column_names = ['Codigo Postal','Lugar', 'Latitud','Longitud']
Quito = pd.DataFrame(columns = column_names)

##Extraer contenido 
code=0
pla=0
lat=0
lon=0
for tr in tr:
    i=0
    for td in tr("td"):  
        if i==0:
            code=td.text
            i+=1
        elif i==1:
            pla=td.text
            i+=1
        elif i==2:
            lat=td.text
            i+=1
        elif i==3:
            lon=td.text
            i+=1
        else:
            continue 
    Quito=Quito.append({'Codigo Postal': code,'Lugar': pla, 'Latitud': lat,'Longitud': lon},ignore_index=True)


Quito= Quito.drop(Quito.index[0])
Quito= Quito[['Codigo Postal','Lugar','Latitud','Longitud']]
Quito['Latitud'] = Quito['Latitud'].astype(float)
Quito['Longitud'] = Quito['Longitud'].astype(float)
print(Quito.head())

  Codigo Postal                    Lugar  Latitud  Longitud
1        170151                 Alangasi -0.30505 -78.41344
2        170101     Alfaro (Chimbacalle) -0.23333 -78.51667
3        170152                 Amaguaña -0.38084 -78.51544
4        170153  Atahualpa (Chabaspamba) -0.18439 -78.49171
5        170129        Belisario Quevedo -0.16563 -78.51045


### Understand the data 
Use a geo map to chek if every single data inside in the beyond data frame is correct locate or have correct latitude or longitude information

In [3]:
##Mapa de barrios de Quito 
# create map of Quito using latitude and longitude values
latitude=-0.225219
longitude= -78.5248
map_quito = folium.Map(location=[latitude, longitude], zoom_start=12)
print(map_quito)

# add markers to map
for lat, lng, borough in zip(Quito['Latitud'], Quito['Longitud'], Quito['Lugar']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quito)  
    
map_quito

### Clean Data 
Correct the latitude and longitud information of wrong data

In [4]:
Quito_df=Quito.replace([-0.27874,-78.55484],[-0.2188216,-78.5135489])
Quito_df=Quito_df.replace([-0.123360, -78.492230],[-0.2956,-78.556])
Quito_df=Quito_df.replace([-0.221970, -78.512390],[-0.1975,-78.4801])
Quito_df=Quito_df.replace( [-0.164340, -78.457750],[-0.1694004,-78.4354042])
Quito_df=Quito_df.replace( [-0.154240, -78.457750],[ -0.1499,-78.4368])
Quito_df=Quito_df.replace( [-0.151010, -78.464380],[0.1713,-78.4112])
Quito_df=Quito_df.replace( [-0.166670, -78.500000],[ -0.1878,-78.4809])
Quito_df=Quito_df.replace([-0.16563, -78.51045],[-0.1085587,-78.4877381])
Quito_df=Quito_df.replace([-0.184390,-78.491710],[0.167761,-78.360507])


Quito_df.head(85)


,Codigo Postal,Lugar,Latitud,Longitud
1,170151,Alangasi,-0.305050,-78.413440
2,170101,Alfaro (Chimbacalle),-0.233330,-78.516670
3,170152,Amaguaña,-0.380840,-78.515440
4,170153,Atahualpa (Chabaspamba),0.167761,-78.360507
5,170129,Belisario Quevedo,-0.108559,-78.487738
6,170102,Benalcazar,-0.182620,-78.481220
7,170154,Calacali,-0.001140,-78.513550
8,170155,Calderon (Carapungo),-0.097490,-78.422510
9,170120,Carcelen,-0.089710,-78.469920
10,170130,Centro Historico,-0.218822,-78.513549


In [5]:
##Mapa de barrios de Quito 
# create map of Quito using latitude and longitude values
latitude=-0.225219
longitude= -78.5248
map_quito = folium.Map(location=[latitude, longitude], zoom_start=10)
print(map_quito)

# add markers to map
for lat, lng, borough in zip(Quito_df['Latitud'], Quito_df['Longitud'], Quito_df['Lugar']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quito)  
    
map_quito

## Foursquare API 

### Foursquare keys 

In [6]:
CLIENT_ID = 'ZHBXLOKXBDJTGTNGXV0QQYJEKWGQIWKIFSTT5J3LIAOR5XRA' # your Foursquare ID
CLIENT_SECRET = 'AMKQBVTDYRHZHFF0BZ1GDYP2JSILCFNH4AYVH0WB5JNFD02V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZHBXLOKXBDJTGTNGXV0QQYJEKWGQIWKIFSTT5J3LIAOR5XRA
CLIENT_SECRET:AMKQBVTDYRHZHFF0BZ1GDYP2JSILCFNH4AYVH0WB5JNFD02V


### Venues extract funtion

In [7]:
 def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    LIMIT=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Lugar', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
# type your answer here

Quito_venues = getNearbyVenues(names=Quito_df['Lugar'],
                                   latitudes=Quito_df['Latitud'],
                                   longitudes=Quito_df['Longitud'],
                                  )

Alangasi
Alfaro (Chimbacalle)
Amaguaña
Atahualpa (Chabaspamba)
Belisario Quevedo
Benalcazar
Calacali
Calderon (Carapungo)
Carcelen
Centro Historico
Chaupicruz (La Concepcion)
Chavezpamba
Checa (Chilpa)
Chilibulo
Chillogallo
Chimbacalle
Cochapamba
Comite Del Pueblo
Condado
Conocoto
Cotocollao
Cumbaya
El Batan
El Beaterio
El Inca
El Quinche
El Salvador
Eloy Alfaro
Gonzalez Suarez
Gualea
Guamani
Guangopolo
Guapulo
Guayllabamba
Iñaquito
Itchimbia
Jipijapa
Kennedy
La Argelia
La Concepción
La Ecuatoriana
La Ferroviaria
La Floresta
La Libertad
La Magdalena
La Mena
La Merced
La Vicentina
Las Cuadras
Llano Chico
Lloa
Mariscal Sucre
Mindo
Nanegal
Nanegalito
Nayón
Nono
Pacto
Pedro Vicente Maldonado
Perucho
Pifo
Pintag
Pomasqui
Ponceano
Puellaro
Puembo
Puengasi
Puerto Quito
Quito
Quitumbe
Rumipamba
San Antonio
San Bartolo
San Blas
San Isidro Del Inca
San Jose De Minas
San Marcos
San Roque
San Sebastian
Santa Barbara
Santa Prisca
Tababela
Tumbaco
Villaflora
Yaruquì
Zàmbiza


In [9]:
print(Quito_venues.shape)
Quito_venues.head(100)

(682, 7)


,Lugar,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alangasi,-0.305050,-78.413440,Parque De Alangasi,-0.307701,-78.415412,Park
1,Alangasi,-0.305050,-78.413440,Cancha Vieja,-0.306081,-78.415789,Soccer Field
2,Alangasi,-0.305050,-78.413440,Cevicheria La Costeñiat,-0.303960,-78.416540,Seafood Restaurant
3,Alfaro (Chimbacalle),-0.233330,-78.516670,Museo Interactivo de Ciencia,-0.236313,-78.516309,Science Museum
4,Alfaro (Chimbacalle),-0.233330,-78.516670,Mesón de la Recoleta,-0.231333,-78.512786,South American Restaurant
5,Belisario Quevedo,-0.108559,-78.487738,"Estadio LIGA Deportiva Universitaria - ""Casa B...",-0.107717,-78.489075,Soccer Stadium
6,Belisario Quevedo,-0.108559,-78.487738,Ceviche con Faldas,-0.105795,-78.490353,Seafood Restaurant
7,Belisario Quevedo,-0.108559,-78.487738,Metrobus: La Ofelia,-0.109602,-78.488360,Bus Station
8,Belisario Quevedo,-0.108559,-78.487738,Terminal Microrregional La Ofelia,-0.110068,-78.488028,Bus Station
9,Belisario Quevedo,-0.108559,-78.487738,Mercado La Ofelia,-0.111389,-78.491014,Farmers Market


### Group by Neighborhoods 

In [10]:
Quito_venues.groupby('Lugar').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Lugar,,,,,,
Alangasi,3,3,3,3,3,3
Alfaro (Chimbacalle),2,2,2,2,2,2
Belisario Quevedo,5,5,5,5,5,5
Benalcazar,67,67,67,67,67,67
Calderon (Carapungo),3,3,3,3,3,3
Carcelen,6,6,6,6,6,6
Centro Historico,45,45,45,45,45,45
Chaupicruz (La Concepcion),15,15,15,15,15,15
Chimbacalle,4,4,4,4,4,4


In [11]:
print('There are {} unique categories.'.format(len(Quito_venues['Venue Category'].unique())))

There are 141 unique categories.


### Create dummies variables of Venue Categories

In [12]:
Quito_onehot = pd.get_dummies(Quito_venues[['Venue Category']], prefix="", prefix_sep="")

Quito_onehot['Lugar'] = Quito_venues['Lugar'] 

fixed_columns = [Quito_onehot.columns[-1]] + list(Quito_onehot.columns[:-1])
Quito_onehot = Quito_onehot[fixed_columns]
Quito_onehot.head(47)

,Lugar,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Bike Shop,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Empanada Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fire Station,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Office,Park,Peruvian Restaurant,Pet Service,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool Hall,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Alangasi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alangasi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alangasi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alfaro (Chimbacalle),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alfaro (Chimbacalle),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Belisario Quevedo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Belisario Quevedo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

In [13]:
Quito_grouped = Quito_onehot.groupby('Lugar').mean().reset_index()
Quito_grouped

,Lugar,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Bike Shop,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Comedy Club,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Empanada Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fire Station,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Office,Park,Peruvian Restaurant,Pet Service,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool Hall,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Alangasi,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Alfaro (Chimbacalle),0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [14]:
num_top_venues = 5
for code in Quito_grouped['Lugar']:
    print("----"+code+"----")
    temp = Quito_grouped[Quito_grouped['Lugar'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alangasi----
                venue  freq
0        Soccer Field  0.33
1                Park  0.33
2  Seafood Restaurant  0.33
3  Advertising Agency  0.00
4         Record Shop  0.00


----Alfaro (Chimbacalle)----
                       venue  freq
0             Science Museum   0.5
1  South American Restaurant   0.5
2                Record Shop   0.0
3                Music Venue   0.0
4                  Nightclub   0.0


----Belisario Quevedo----
                venue  freq
0         Bus Station   0.4
1      Soccer Stadium   0.2
2      Farmers Market   0.2
3  Seafood Restaurant   0.2
4  Advertising Agency   0.0


----Benalcazar----
                venue  freq
0  Italian Restaurant  0.10
1               Hotel  0.06
2              Bakery  0.06
3         Coffee Shop  0.06
4      Ice Cream Shop  0.04


----Calderon (Carapungo)----
                venue  freq
0                Park  0.33
1  Chinese Restaurant  0.33
2         Pizza Place  0.33
3  Advertising Agency  0.00
4            Pharm

                venue  freq
0               Plaza   1.0
1  Advertising Agency   0.0
2            Pharmacy   0.0
3         Music Venue   0.0
4           Nightclub   0.0


----Puembo----
                venue  freq
0   Recreation Center   0.5
1  Athletics & Sports   0.5
2            Pharmacy   0.0
3              Museum   0.0
4         Music Venue   0.0


----Puengasi----
                venue  freq
0         Bus Station  0.75
1                Park  0.25
2  Advertising Agency  0.00
3   Recreation Center  0.00
4         Music Venue  0.00


----Quito----
                venue  freq
0       Historic Site   0.5
1    Asian Restaurant   0.5
2  Advertising Agency   0.0
3            Pharmacy   0.0
4         Music Venue   0.0


----Quitumbe----
                  venue  freq
0           Bus Station  0.50
1          Fire Station  0.25
2  Fast Food Restaurant  0.25
3              Pharmacy  0.00
4             Nightclub  0.00


----Rumipamba----
              venue  freq
0          Pharmacy  0.18
1    

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### More common Venues in each neighborhood

In [16]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Lugar']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Lugar'] = Quito_grouped['Lugar']

for ind in np.arange(Quito_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Quito_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Lugar,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alangasi,Park,Soccer Field,Seafood Restaurant,Food & Drink Shop,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market,Farm,Wings Joint
1,Alfaro (Chimbacalle),South American Restaurant,Science Museum,Comedy Club,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market,Wings Joint
2,Belisario Quevedo,Bus Station,Soccer Stadium,Seafood Restaurant,Farmers Market,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant,Wings Joint
3,Benalcazar,Italian Restaurant,Hotel,Bakery,Coffee Shop,Ice Cream Shop,French Restaurant,Japanese Restaurant,Fast Food Restaurant,Dessert Shop,Movie Theater
4,Calderon (Carapungo),Pizza Place,Chinese Restaurant,Park,Wings Joint,Farm,Food,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market
5,Carcelen,BBQ Joint,Gym,Seafood Restaurant,Restaurant,Soccer Field,Pizza Place,Department Store,Deli / Bodega,Dessert Shop,Diner
6,Centro Historico,Hotel,Restaurant,Shopping Mall,Coffee Shop,Historic Site,Diner,History Museum,Breakfast Spot,Plaza,Ice Cream Shop
7,Chaupicruz (La Concepcion),Restaurant,Pizza Place,Chinese Restaurant,Mexican Restaurant,Fried Chicken Joint,Bar,Latin American Restaurant,Seafood Restaurant,Burger Joint,Asian Restaurant
8,Chimbacalle,Print Shop,Bus Station,Diner,Stadium,Farmers Market,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant
9,Cochapamba,Toy / Game Store,Sporting Goods Shop,Record Shop,Ice Cream Shop,Department Store,Supermarket,Sandwich Place,Furniture / Home Store,Wings Joint,Fire Station


In [17]:

from sklearn.cluster import KMeans

kclusters = 5

Quito_grouped_clustering = Quito_grouped.drop('Lugar', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering)

kmeans.labels_

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0,
       4, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       3, 0, 1, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

### Map of Venue

In [20]:

body = client_bfb6a1fe170c49c5ad9e6879e865a633.get_object(Bucket='pythonbasictodatascienceproject-donotdelete-pr-4fx4c2v48kskhx',Key='Coord.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_coord = pd.read_excel(body)
df_data_coord


,Lugar,Latitud,Longitud
0,Alangasi,-0.305050,-78.413440
1,Alfaro (Chimbacalle),-0.233330,-78.516670
2,Amaguaña,-0.380840,-78.515440
3,Belisario Quevedo,-0.108559,-78.487738
4,Benalcazar,-0.182620,-78.481220
5,Calderon (Carapungo),-0.097490,-78.422510
6,Carcelen,-0.089710,-78.469920
7,Centro Historico,-0.180700,-78.467800
8,Chaupicruz (La Concepcion),-0.157490,-78.486970
9,Chimbacalle,-0.248200,-78.515660


In [23]:
Quito_merged = df_data_coord[0:64]

Quito_merged['Cluster Labels'] = kmeans.labels_

Quito_merged.head(64)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Lugar,Latitud,Longitud,Cluster Labels
0,Alangasi,-0.305050,-78.413440,0
1,Alfaro (Chimbacalle),-0.233330,-78.516670,0
2,Amaguaña,-0.380840,-78.515440,3
3,Belisario Quevedo,-0.108559,-78.487738,0
4,Benalcazar,-0.182620,-78.481220,0
5,Calderon (Carapungo),-0.097490,-78.422510,0
6,Carcelen,-0.089710,-78.469920,0
7,Centro Historico,-0.180700,-78.467800,0
8,Chaupicruz (La Concepcion),-0.157490,-78.486970,0
9,Chimbacalle,-0.248200,-78.515660,0


In [24]:
Quito_merged = Quito_merged.join(neighborhoods_venues_sorted.set_index('Lugar'), on='Lugar')

Quito_merged.head(65)

,Lugar,Latitud,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alangasi,-0.305050,-78.413440,0,Park,Soccer Field,Seafood Restaurant,Food & Drink Shop,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market,Farm,Wings Joint
1,Alfaro (Chimbacalle),-0.233330,-78.516670,0,South American Restaurant,Science Museum,Comedy Club,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market,Wings Joint
2,Amaguaña,-0.380840,-78.515440,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Belisario Quevedo,-0.108559,-78.487738,0,Bus Station,Soccer Stadium,Seafood Restaurant,Farmers Market,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant,Wings Joint
4,Benalcazar,-0.182620,-78.481220,0,Italian Restaurant,Hotel,Bakery,Coffee Shop,Ice Cream Shop,French Restaurant,Japanese Restaurant,Fast Food Restaurant,Dessert Shop,Movie Theater
5,Calderon (Carapungo),-0.097490,-78.422510,0,Pizza Place,Chinese Restaurant,Park,Wings Joint,Farm,Food,Flea Market,Fire Station,Fast Food Restaurant,Farmers Market
6,Carcelen,-0.089710,-78.469920,0,BBQ Joint,Gym,Seafood Restaurant,Restaurant,Soccer Field,Pizza Place,Department Store,Deli / Bodega,Dessert Shop,Diner
7,Centro Historico,-0.180700,-78.467800,0,Hotel,Restaurant,Shopping Mall,Coffee Shop,Historic Site,Diner,History Museum,Breakfast Spot,Plaza,Ice Cream Shop
8,Chaupicruz (La Concepcion),-0.157490,-78.486970,0,Restaurant,Pizza Place,Chinese Restaurant,Mexican Restaurant,Fried Chicken Joint,Bar,Latin American Restaurant,Seafood Restaurant,Burger Joint,Asian Restaurant
9,Chimbacalle,-0.248200,-78.515660,0,Print Shop,Bus Station,Diner,Stadium,Farmers Market,Food & Drink Shop,Food,Flea Market,Fire Station,Fast Food Restaurant


In [25]:
map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged['Latitud'], Quito_merged['Longitud'], Quito_merged['Lugar'], Quito_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


# Import extra data for the Clasification of Places in Quito

### Socio-economic Indicators 

In [19]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bfb6a1fe170c49c5ad9e6879e865a633 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='d3yhoCFyeMFHRcVIiglNccdPTUgoHl9rPt7iCuI2VXvU',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bfb6a1fe170c49c5ad9e6879e865a633.get_object(Bucket='pythonbasictodatascienceproject-donotdelete-pr-4fx4c2v48kskhx',Key='Base_Parroquias.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
print("Shape of Socio econmic information data frame:", df_data_0.shape)
df_data_0.head(65)


Shape of Socio econmic information data frame: (65, 67)


,Parroquias,Poblacion,Densidad,Proporción_Mujer,Promedio_edad,Promedio_per_hogar,Madres_Adolscentes,PEA,PEA_%,"Agricultura, ganaderia, silvicultura y pesca",Explotación de minas y canteras,Industrias manufactureras,"Suministro de electricidad, gas, vapor y aire acondicionado","Distribución de agua, alcantarillado y gestión de desechos",Construcción,Comercio al por mayor y menor,Transporte y almacenamiento,Actividades de alojamiento y servicio de comidas,Información y comunicación,Actividades financieras y de seguros,Actividades inmobiliarias,"Actividades profesionales, cientificas y tecnicas",Actividades de servicios administrativos y de apoyo,Administración publica y defensa,Enseñanza,Actividades de la atención de la salud humana,"Artes, entretenimiento y recreacion",Otras actividades de servicios,Actividades de los hogares como empleadores,Actividades de organizaciones y organos extraterritoriales,No declarado,Trabajador nuevo,"Empleado/a u obrero/a del Estado, Gobierno, Municipio, Consejo Provincial, Juntas Parroquiales",Empleado/a u obrero/a privado,Jornalero/a o peón,Patron/a,Socio/a,Cuenta propia,Trabajador/a no remunerado,Empleado/a doméstico/a,NA,Alfabetismo,Años_Escolaridad,Asistencia_basica,Asistencia_Bachiller,Asistencia_Superior,Edu_Privada,Analfabetismo_digital,Vienda_Mal,Vivienda_Buen,Agua_potable,Viviendas_inadecuadas,Salubridad_inadecuadas,Madres_Solteras,Discapacidad,Indígena,Afroecuatoriano/a,Negro/a,Mulato/a,Montubio/a,Mestizo/a,Blanco/a,Otro/a,Seguro_Privado,IESS_campesino,IESS,Jubilados
0,Alangasí,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Amaguaña,31106,44.363608,0.505079,28.307336,3.830000,0.032300,14158,0.455153,0.090463,0.003003,0.254248,0.002124,0.003296,0.080794,0.155435,0.045341,0.038383,0.010328,0.009449,0.002051,0.019851,0.030179,0.047026,0.036991,0.020070,0.006373,0.021682,0.061163,0.000000,0.061749,0,0.101963,0.492895,0.062482,0.026004,0.008717,0.193598,0.013185,0.062189,0.038969,0.993509,9.110869,0.969914,0.845095,0.301080,0.273414,0.251010,0.029371,0.424822,0.815523,0.085461,0.139845,0.147415,0.023147,0.019385,0.019064,0.003343,0.010384,0.010127,0.909149,0.026812,0.001736,0.047515,0.001245,0.436346,0.302326
2,Atahualpa,1901,15.200114,0.501841,34.276696,3.358657,0.044444,840,840.000000,0.533333,0.002424,0.033939,0.007273,0.001212,0.064242,0.065455,0.032727,0.010909,0.006061,0.009697,0.000000,0.007273,0.014545,0.029091,0.031515,0.004848,0.001212,0.012121,0.048485,0.000000,0.083636,0,0.095758,0.215758,0.241212,0.006061,0.004848,0.322424,0.007273,0.054545,0.052121,0.986971,7.467333,0.953039,0.722222,0.191045,0.085106,0.394571,0.066071,0.276786,0.667857,0.132509,0.367491,0.183893,0.063125,0.005260,0.024724,0.000526,0.012625,0.005786,0.875855,0.073119,0.002104,0.026302,0.344242,0.172121,0.147368
3,Belisario Quevedo,45370,8345.680223,0.527551,32.628675,3.100000,0.022811,24008,24008.000000,0.007468,0.005579,0.103585,0.003602,0.002943,0.049157,0.193419,0.035539,0.057811,0.036637,0.029257,0.006282,0.071824,0.050650,0.073405,0.078677,0.062379,0.013574,0.031146,0.042040,0.001669,0.043358,0,0.165568,0.508039,0.015771,0.053462,0.017923,0.159682,0.014057,0.043973,0.021525,0.997355,12.262085,0.979246,0.892808,0.454538,0.526198,0.116538,0.007102,0.582747,0.945456,0.014964,0.033686,0.209690,0.022636,0.020785,0.019154,0.005047,0.015759,0.010271,0.823672,0.099471,0.005841,0.128477,0.001318,0.530531,0.390399
4,Calacalí,3895,48.769643,0.500128,30.433376,3.720000,0.039000,1808,0.464185,0.274027

### Geo-Information

In [26]:

body = client_bfb6a1fe170c49c5ad9e6879e865a633.get_object(Bucket='pythonbasictodatascienceproject-donotdelete-pr-4fx4c2v48kskhx',Key='Coordenadas_parroquias.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_Quito = pd.read_excel(body)
df_Quito


,Parroquias,Latitud,Longitud
0,Alangasí,-0.305050,-78.413440
1,Amaguaña,-0.380840,-78.515440
2,Atahualpa,0.167761,-78.360507
3,Belisario Quevedo,-0.108559,-78.487738
4,Calacalí,-0.001140,-78.513550
5,Calderon,-0.097490,-78.422510
6,Carcelen,-0.089710,-78.469920
7,Centro Historico,-0.180700,-78.467800
8,Chavezpamba,0.133330,-78.383330
9,Checa,-0.126640,-78.311730


### Clustering by socio economic indicators

### Education Skills 

In [27]:

from sklearn.cluster import KMeans

kclusters = 5
Quito_gr=df_data_0.filter(['Parroquias','Alfabetismo','Años_Escolaridad','Asistencia_basica','Asistencia_Bachiller','Asistencia_Superior'])
Quito_grouped_clustering = df_data_0.filter(['Alfabetismo','Años_Escolaridad','Asistencia_basica','Asistencia_Bachiller','Asistencia_Superior'])
Quito_gr.head()



,Parroquias,Alfabetismo,Años_Escolaridad,Asistencia_basica,Asistencia_Bachiller,Asistencia_Superior
0,Alangasí,0.000000,0.000000,0.000000,0.000000,0.000000
1,Amaguaña,0.993509,9.110869,0.969914,0.845095,0.301080
2,Atahualpa,0.986971,7.467333,0.953039,0.722222,0.191045
3,Belisario Quevedo,0.997355,12.262085,0.979246,0.892808,0.454538
4,Calacalí,0.981928,8.243497,0.957659,0.792627,0.218310


In [28]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering)

kmeans.labels_

array([3, 0, 2, 1, 0, 4, 4, 4, 2, 2, 4, 0, 4, 4, 0, 1, 4, 1, 1, 0, 0, 2,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 4, 0, 0, 2, 1, 2, 2, 4, 2, 2,
       2, 0, 0, 1, 4, 2, 0, 4, 0, 1, 0, 4, 4, 2, 4, 4, 0, 4, 0, 0, 0],
      dtype=int32)

In [29]:
Quito_merged = df_Quito[0:65]

Quito_merged['Cluster Labels'] = kmeans.labels_

Quito_merged.head(65)

Quito_merged = Quito_merged.join(Quito_gr.set_index('Parroquias'), on= 'Parroquias')

Quito_merged.head(65)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Parroquias,Latitud,Longitud,Cluster Labels,Alfabetismo,Años_Escolaridad,Asistencia_basica,Asistencia_Bachiller,Asistencia_Superior
0,Alangasí,-0.305050,-78.413440,3,0.000000,0.000000,0.000000,0.000000,0.000000
1,Amaguaña,-0.380840,-78.515440,0,0.993509,9.110869,0.969914,0.845095,0.301080
2,Atahualpa,0.167761,-78.360507,2,0.986971,7.467333,0.953039,0.722222,0.191045
3,Belisario Quevedo,-0.108559,-78.487738,1,0.997355,12.262085,0.979246,0.892808,0.454538
4,Calacalí,-0.001140,-78.513550,0,0.981928,8.243497,0.957659,0.792627,0.218310
5,Calderon,-0.097490,-78.422510,4,0.991250,9.825857,0.967014,0.829655,0.295303
6,Carcelen,-0.089710,-78.469920,4,0.996927,11.316247,0.978325,0.874728,0.362745
7,Centro Historico,-0.180700,-78.467800,4,0.991732,10.065839,0.965633,0.794551,0.315650
8,Chavezpamba,0.133330,-78.383330,2,0.992063,7.211940,0.933735,0.761905,0.231343
9,Checa,-0.126640,-78.311730,2,0.991629,7.965808,0.956714,0.743501,0.194009


In [30]:

map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged['Latitud'], Quito_merged['Longitud'], Quito_merged['Parroquias'], Quito_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


## Work Sector 

In [15]:
from sklearn.cluster import KMeans

kclusters = 5
Quito_gr_work=df_data_0.filter(['Parroquias','Agricultura, ganaderia, silvicultura y pesca','Explotación de minas y canteras', 'Industrias manufactureras','Suministro de electricidad, gas, vapor y aire acondicionado','Distribución de agua, alcantarillado y gestión de desechos','Construcción','Comercio al por mayor y menor','Transporte y almacenamiento','Actividades de alojamiento y servicio de comidas','Información y comunicación','Actividades financieras y de seguros','Actividades inmobiliarias','Actividades profesionales, cientificas y tecnicas','Actividades de servicios administrativos y de apoyo','Administración publica y defensa','Enseñanza','Actividades de la atención de la salud humana','Artes, entretenimiento y recreacion','Otras actividades de servicios','Actividades de los hogares como empleadores','Actividades de organizaciones y organos extraterritoriales','No declarado','Trabajador nuevo'])
Quito_grouped_clustering_work = df_data_0.filter(['Agricultura, ganaderia, silvicultura y pesca','Explotación de minas y canteras', 'Industrias manufactureras','Suministro de electricidad, gas, vapor y aire acondicionado','Distribución de agua, alcantarillado y gestión de desechos','Construcción','Comercio al por mayor y menor','Transporte y almacenamiento','Actividades de alojamiento y servicio de comidas','Información y comunicación','Actividades financieras y de seguros','Actividades inmobiliarias','Actividades profesionales, cientificas y tecnicas','Actividades de servicios administrativos y de apoyo','Administración publica y defensa','Enseñanza','Actividades de la atención de la salud humana','Artes, entretenimiento y recreacion','Otras actividades de servicios','Actividades de los hogares como empleadores','Actividades de organizaciones y organos extraterritoriales','No declarado','Trabajador nuevo'])
print(Quito_grouped_clustering_work)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering_work)

kmeans.labels_

    Agricultura, ganaderia, silvicultura y pesca  \
0                                       0.000000   
1                                       0.090463   
2                                       0.533333   
3                                       0.007468   
4                                       0.274027   
5                                       0.020964   
6                                       0.011544   
7                                       0.006451   
8                                       0.553936   
9                                       0.408327   
10                                      0.008918   
11                                      0.016289   
12                                      0.006011   
13                                      0.010696   
14                                      0.009113   
15                                      0.009677   
16                                      0.018453   
17                                      0.010364   
18          

array([3, 1, 0, 3, 2, 4, 4, 4, 0, 2, 4, 4, 3, 4, 4, 3, 3, 3, 3, 1, 2, 0,
       4, 1, 2, 3, 3, 3, 3, 4, 4, 4, 4, 3, 4, 1, 1, 0, 3, 0, 2, 3, 0, 0,
       0, 1, 1, 3, 4, 0, 2, 4, 4, 3, 1, 4, 4, 0, 4, 4, 2, 1, 4, 2, 1],
      dtype=int32)

In [16]:
Quito_merged_work = df_Quito[0:65]

Quito_merged_work['Cluster Labels'] = kmeans.labels_

Quito_merged_work.head(65)

Quito_merged_work = Quito_merged_work.join(Quito_gr_work.set_index('Parroquias'), on= 'Parroquias')

Quito_merged_work.head(65)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Parroquias,Latitud,Longitud,Cluster Labels,"Agricultura, ganaderia, silvicultura y pesca",Explotación de minas y canteras,Industrias manufactureras,"Suministro de electricidad, gas, vapor y aire acondicionado","Distribución de agua, alcantarillado y gestión de desechos",Construcción,Comercio al por mayor y menor,Transporte y almacenamiento,Actividades de alojamiento y servicio de comidas,Información y comunicación,Actividades financieras y de seguros,Actividades inmobiliarias,"Actividades profesionales, cientificas y tecnicas",Actividades de servicios administrativos y de apoyo,Administración publica y defensa,Enseñanza,Actividades de la atención de la salud humana,"Artes, entretenimiento y recreacion",Otras actividades de servicios,Actividades de los hogares como empleadores,Actividades de organizaciones y organos extraterritoriales,No declarado,Trabajador nuevo
0,Alangasí,-0.305050,-78.413440,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Amaguaña,-0.380840,-78.515440,1,0.090463,0.003003,0.254248,0.002124,0.003296,0.080794,0.155435,0.045341,0.038383,0.010328,0.009449,0.002051,0.019851,0.030179,0.047026,0.036991,0.020070,0.006373,0.021682,0.061163,0.000000,0.061749,0.0
2,Atahualpa,-1.055272,-80.459721,0,0.533333,0.002424,0.033939,0.007273,0.001212,0.064242,0.065455,0.032727,0.010909,0.006061,0.009697,0.000000,0.007273,0.014545,0.029091,0.031515,0.004848,0.001212,0.012121,0.048485,0.000000,0.083636,0.0
3,Belisario Quevedo,-0.108559,-78.487738,3,0.007468,0.005579,0.103585,0.003602,0.002943,0.049157,0.193419,0.035539,0.057811,0.036637,0.029257,0.006282,0.071824,0.050650,0.073405,0.078677,0.062379,0.013574,0.031146,0.042040,0.001669,0.043358,0.0
4,Calacalí,-0.001140,-78.513550,2,0.274027,0.003432,0.144165,0.001144,0.000572,0.085812,0.125286,0.044050,0.036041,0.007437,0.006293,0.002288,0.023455,0.037185,0.037185,0.022311,0.016018,0.003432,0.015446,0.068650,0.000000,0.045767,0.0
5,Calderon,-0.097490,-78.422510,4,0.020964,0.003963,0.180647,0.002656,0.007108,0.068477,0.207871,0.071708,0.048504,0.021452,0.015436,0.004609,0.036055,0.050629,0.039731,0.045661,0.028990,0.008572,0.024999,0.057004,0.000517,0.054448,0.0
6,Carcelen,-0.089710,-78.469920,4,0.011544,0.007557,0.139629,0.002886,0.004101,0.047581,0.209425,0.060454,0.049935,0.030569,0.029961,0.004139,0.051644,0.051037,0.065429,0.058404,0.041657,0.008620,0.026240,0.043556,0.001101,0.054530,0.0
7,Centro Historico,-0.180700,-78.467800,4,0.006451,0.001886,0.108426,0.001935,0.004019,0.053841,0.314113,0.032503,0.083615,0.020792,0.015284,0.003275,0.039152,0.059150,0.049672,0.047936,0.034538,0.012009,0.029228,0.038160,0.000198,0.043817,0.0
8,Chavezpamba,0.133330,-78.383330,0,0.553936,0.002915,0.011662,0.000000,0.011662,0.037901,0.064140,0.026239,0.008746,0.002915,0.002915,0.000000,0.005831,0.011662,0.026239,0.026239,0.017493,0.000000,0.014577,0.014577,0.000000,0.160350,0.0
9,Checa,-0.126640,-78.311730,2,0.408327,0.001217,0.098856,0.002435,0.003409,0.093012,0.100804,0.031653,0.023375,0.005113,0.002435,0.001704,0.010713,0.032871,0.011200,0.017044,0.009983,0.002922,0.009983,0.043097,0.000000,0.089847,0.0


In [17]:
map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged_work['Latitud'], Quito_merged_work['Longitud'], Quito_merged_work['Parroquias'], Quito_merged_work['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


## Living Conditions

In [31]:
from sklearn.cluster import KMeans
kclusters = 5
Quito_lv=df_data_0.filter(['Parroquias','Vienda_Mal','Vivienda_Buen ','Agua_potable ','Viviendas_inadecuadas ','Salubridad_inadecuadas '])
Quito_grouped_clustering_living = df_data_0.filter(['Vienda_Mal','Vivienda_Buen ','Agua_potable ','Viviendas_inadecuadas ','Salubridad_inadecuadas '])
Quito_lv.head()
Quito_lv.describe()

,Vienda_Mal,Vivienda_Buen,Agua_potable,Viviendas_inadecuadas,Salubridad_inadecuadas
count,65.000000,65.000000,65.000000,65.000000,65.000000
mean,0.022401,0.473362,0.870581,0.043684,0.108248
std,0.028116,0.165461,0.194956,0.048979,0.135402
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.006119,0.378712,0.801762,0.010684,0.010187
50%,0.010608,0.476454,0.970015,0.023756,0.041494
75%,0.026346,0.587589,0.990027,0.054545,0.173417
max,0.128801,0.808484,0.997718,0.195695,0.614481


In [32]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering_living)

kmeans.labels_

array([0, 4, 3, 2, 4, 1, 2, 1, 4, 4, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 3,
       1, 4, 4, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 4, 1, 3, 2, 3, 3, 2, 3, 3,
       4, 4, 4, 2, 2, 3, 1, 2, 2, 2, 1, 2, 2, 3, 1, 2, 1, 1, 1, 4, 1],
      dtype=int32)

In [33]:
Quito_merged_living = df_Quito[0:65]

Quito_merged_living['Cluster Labels'] = kmeans.labels_

Quito_merged_living.head(65)

Quito_merged_living = Quito_merged_living.join(Quito_lv.set_index('Parroquias'), on= 'Parroquias')

Quito_merged_living.head(65)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Parroquias,Latitud,Longitud,Cluster Labels,Vienda_Mal,Vivienda_Buen,Agua_potable,Viviendas_inadecuadas,Salubridad_inadecuadas
0,Alangasí,-0.305050,-78.413440,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Amaguaña,-0.380840,-78.515440,4,0.029371,0.424822,0.815523,0.085461,0.139845
2,Atahualpa,0.167761,-78.360507,3,0.066071,0.276786,0.667857,0.132509,0.367491
3,Belisario Quevedo,-0.108559,-78.487738,2,0.007102,0.582747,0.945456,0.014964,0.033686
4,Calacalí,-0.001140,-78.513550,4,0.037500,0.322115,0.715385,0.054545,0.216268
5,Calderon,-0.097490,-78.422510,1,0.007978,0.555702,0.984678,0.019500,0.041494
6,Carcelen,-0.089710,-78.469920,2,0.004024,0.649945,0.982802,0.010243,0.010307
7,Centro Historico,-0.180700,-78.467800,1,0.014111,0.392196,0.987544,0.016848,0.010187
8,Chavezpamba,0.133330,-78.383330,4,0.085470,0.230769,0.901709,0.172996,0.295359
9,Checa,-0.126640,-78.311730,4,0.024718,0.347788,0.782741,0.129558,0.202488


In [34]:
map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged_living['Latitud'], Quito_merged_living['Longitud'], Quito_merged_living['Parroquias'], Quito_merged_living['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


### Health coverage

In [36]:
from sklearn.cluster import KMeans
kclusters = 5
Quito_sec=df_data_0.filter(['Parroquias',"Seguro_Privado","IESS","IESS_campesino","Jubilados"])
Quito_grouped_clustering_sec = df_data_0.filter(['Vienda_Mal',"Seguro_Privado","IESS","IESS_campesino ","Jubilados"])
Quito_sec.head()
Quito_sec.describe()

,Seguro_Privado,IESS,IESS_campesino,Jubilados
count,65.000000,65.000000,65.000000,65.000000
mean,0.076722,0.392331,0.029802,0.252634
std,0.061894,0.137497,0.080186,0.125214
min,0.000000,0.000000,0.000000,0.000000
25%,0.032449,0.304348,0.001137,0.147368
50%,0.056315,0.396817,0.001618,0.260870
75%,0.103671,0.499699,0.003045,0.362330
max,0.266710,0.602165,0.365087,0.469173


In [37]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering_sec)

kmeans.labels_

array([4, 2, 4, 1, 0, 2, 3, 2, 0, 0, 2, 2, 1, 2, 2, 3, 1, 3, 3, 2, 0, 4,
       0, 0, 0, 3, 1, 3, 3, 2, 2, 2, 0, 1, 2, 0, 2, 4, 3, 4, 0, 1, 4, 4,
       0, 0, 0, 3, 1, 0, 2, 1, 2, 3, 2, 1, 1, 0, 2, 1, 2, 2, 2, 0, 2],
      dtype=int32)

In [38]:
Quito_merged_sec = df_Quito[0:65]

Quito_merged_sec['Cluster Labels'] = kmeans.labels_

Quito_merged_sec.head(65)

Quito_merged_sec = Quito_merged_sec.join(Quito_sec.set_index('Parroquias'), on= 'Parroquias')

Quito_merged_sec.head(65)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Parroquias,Latitud,Longitud,Cluster Labels,Seguro_Privado,IESS,IESS_campesino,Jubilados
0,Alangasí,-0.305050,-78.413440,4,0.000000,0.000000,0.000000,0.000000
1,Amaguaña,-0.380840,-78.515440,2,0.047515,0.436346,0.001245,0.302326
2,Atahualpa,0.167761,-78.360507,4,0.026302,0.172121,0.344242,0.147368
3,Belisario Quevedo,-0.108559,-78.487738,1,0.128477,0.530531,0.001318,0.390399
4,Calacalí,-0.001140,-78.513550,0,0.046213,0.304348,0.006293,0.113772
5,Calderon,-0.097490,-78.422510,2,0.072306,0.452530,0.002599,0.272266
6,Carcelen,-0.089710,-78.469920,3,0.127198,0.548568,0.001747,0.463787
7,Centro Historico,-0.180700,-78.467800,2,0.060814,0.366564,0.002680,0.245350
8,Chavezpamba,0.133330,-78.383330,0,0.031211,0.276968,0.239067,0.075472
9,Checa,-0.126640,-78.311730,0,0.028619,0.376674,0.007061,0.168224


In [67]:
map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged_sec['Latitud'], Quito_merged_sec['Longitud'], Quito_merged_sec['Parroquias'], Quito_merged_sec['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


### Teen Pregnancy

In [69]:
from sklearn.cluster import KMeans
kclusters = 5
Quito_teen=df_data_0.filter(['Parroquias','Madres_Adolscentes'])
Quito_grouped_clustering_teen = df_data_0.filter(['Vienda_Mal','Madres_Adolscentes'])
Quito_teen.head()
Quito_teen.describe()

,Madres_Adolscentes
count,65.000000
mean,0.035410
std,0.017661
min,0.000000
25%,0.024100
50%,0.034800
75%,0.041500
max,0.114754


In [70]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quito_grouped_clustering_teen)

kmeans.labels_

array([2, 0, 3, 2, 3, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 4,
       0, 3, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 3, 2, 1, 4,
       0, 0, 3, 2, 0, 3, 0, 0, 0, 2, 2, 2, 0, 4, 0, 2, 0, 0, 0, 0, 0],
      dtype=int32)

In [71]:
Quito_merged_teen = df_Quito[0:65]

Quito_merged_teen['Cluster Labels'] = kmeans.labels_

Quito_merged_teen.head(65)

Quito_merged_teen = Quito_merged_teen.join(Quito_teen.set_index('Parroquias'), on= 'Parroquias')

Quito_merged_teen.head(65)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Parroquias,Latitud,Longitud,Cluster Labels,Madres_Adolscentes
0,Alangasí,-0.305050,-78.413440,2,0.000000
1,Amaguaña,-0.380840,-78.515440,0,0.032300
2,Atahualpa,0.167761,-78.360507,3,0.044444
3,Belisario Quevedo,-0.108559,-78.487738,2,0.022811
4,Calacalí,-0.001140,-78.513550,3,0.039000
5,Calderon,-0.097490,-78.422510,0,0.033900
6,Carcelen,-0.089710,-78.469920,2,0.028000
7,Centro Historico,-0.180700,-78.467800,0,0.038500
8,Chavezpamba,0.133330,-78.383330,1,0.114754
9,Checa,-0.126640,-78.311730,0,0.043900


In [72]:
map_clusters = folium.Map(location=[-0.225219, -78.5248], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Quito_merged_teen['Latitud'], Quito_merged_teen['Longitud'], Quito_merged_teen['Parroquias'], Quito_merged_teen['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']
